In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

import os
from pycocotools.coco import COCO

In [2]:
from utils.YoloCocoDataset import YOLO_COCO_DATASET
from torchvision.transforms.v2 import Compose, ToImage, RandomResizedCrop, ColorJitter

dataDir='/home/awatras/Datasets/cocoapi'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

tf = Compose([ToImage(), RandomResizedCrop(size=(448,448)), ColorJitter(brightness = 0.5, contrast = 0, saturation = 0.5, hue = 0.0)])

ds = YOLO_COCO_DATASET('/home/awatras/Datasets/cocoapi/images/val2017', annFile, transform=tf)

loading annotations into memory...
Done (t=1.30s)
creating index...
index created!


In [ ]:
from backbone.yolov1 import YoloV1Model
from utils.losses import YoloV1Loss

learning_rate = 1e-3
batch_size = 64
epochs = 5

model = YoloV1Model().to('cuda')
loss_fn = YoloV1Loss(7,2,10)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
print(model)

YoloV1Model(
  (conv_layers): Sequential(
    (0): CNNBlock(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): CNNBlock(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): CNNBlock(
      (conv): Conv2d(192, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (5)

In [ ]:
dl = DataLoader(ds, 64, True)

for x,y in dl:
    x = x.to('cuda')
    y = y.to('cuda')

    y_hat = model(x)
    loss = loss(y_hat, y)
    print(loss)
    break


torch.Size([64, 4410])
